In [2]:
using DataFrames, DataFramesMeta, LinearAlgebra, Distributions, Optim, Revise, Plots, CSV
using JuMP, Ipopt
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__)
using BLPmodule; const m = BLPmodule;

┌ Info: Precompiling BLPmodule [top-level]
└ @ Base loading.jl:1317


In [ ]:
m.gmm(β, dfj.s, X, Z, ζ)

(X' * X )*( X'  * deltas)
# model = Model(Ipopt.Optimizer)
# @variable(model, β)
# @NLobjective(model, Min, m.gmm(β, dfj.s, X, Z, ζ))

---
# this is messed up
# model = Model(Ipopt.Optimizer)
# @variable(model, β)
# register(model, :g, 1, m.gmm, m.grad, autodiff = true)
# @NLobjective(model, Min, (s, X, Z, ζ))
# optimize!(model)
# solution_summary(model; verbose = true)